In [104]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
import numpy as np

Bank Marketing (with social/economic context) dataset with loan target variable

Load data

About this file
It is a dataset that describing Portugal bank marketing campaigns results.
Conducted campaigns were based mostly on direct phone calls, offering bank client to place a term deposit.
If after all marking afforts client had agreed to place deposit - target variable marked 'yes', otherwise 'no'

In [105]:
df = pd.read_csv('../data/raw/bank-additional-full.csv',sep=';')
df = df.drop(['duration'],axis=1)

Checking dataset

In [106]:
df.shape

(41188, 20)

In [107]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [108]:
columns = list(df.columns)

In [109]:
columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

Renaming cols

Checking missing values

In [110]:
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [111]:
df.age.describe()

count    41188.00000
mean        40.02406
std         10.42125
min         17.00000
25%         32.00000
50%         38.00000
75%         47.00000
max         98.00000
Name: age, dtype: float64

In [112]:
df.y.value_counts()/df.shape[0]*100  

no     88.734583
yes    11.265417
Name: y, dtype: float64

In [113]:
df.y.replace({"yes":1, "no":0}, inplace=True) # target convert to int

In [114]:
df.job.value_counts()/df.shape[0]*100  

admin.           25.303486
blue-collar      22.467709
technician       16.371273
services          9.636302
management        7.099155
retired           4.175974
entrepreneur      3.535010
self-employed     3.450034
housemaid         2.573565
unemployed        2.461882
student           2.124405
unknown           0.801204
Name: job, dtype: float64

In [115]:
#df['job'] = df['job'].replace('unknown',np.nan) # maybe delete unknown

In [116]:
df.marital.value_counts()/df.shape[0]*100 

married     60.522482
single      28.085850
divorced    11.197436
unknown      0.194231
Name: marital, dtype: float64

In [117]:
#df['marital'] = df['marital'].replace('unknown',np.nan) # maybe delete unknown because of small samples

In [118]:
#df = df.dropna()

In [119]:
df.education.value_counts()/df.shape[0]*100 

university.degree      29.542585
high.school            23.101389
basic.9y               14.676605
professional.course    12.729436
basic.4y               10.138875
basic.6y                5.564728
unknown                 4.202680
illiterate              0.043702
Name: education, dtype: float64

In [120]:
#df['education'] = df['education'].replace('unknown','university.degree') # possible option replace with most common

In [121]:
df.default.value_counts()/df.shape[0]*100 

no         79.120132
unknown    20.872584
yes         0.007284
Name: default, dtype: float64

In [122]:
df = df.drop(['default'],axis=1) # delete column because too much missing and almost no yes

In [123]:
df.housing.value_counts()/df.shape[0]*100

yes        52.384190
no         45.212198
unknown     2.403613
Name: housing, dtype: float64

In [124]:
df.groupby(["housing","y"]).education.count()/df.shape[0]*100

housing  y
no       0    40.293289
         1     4.918908
unknown  0     2.143828
         1     0.259784
yes      0    46.297465
         1     6.086724
Name: education, dtype: float64

In [125]:
df.loan.value_counts()/df.shape[0]*100

no         82.426920
yes        15.169467
unknown     2.403613
Name: loan, dtype: float64

In [126]:
df.groupby(["loan","y"]).education.count()/df.shape[0]*100

loan     y
no       0    73.079538
         1     9.347383
unknown  0     2.143828
         1     0.259784
yes      0    13.511217
         1     1.658250
Name: education, dtype: float64

In [127]:
df.contact.value_counts()/df.shape[0]*100

cellular     63.474798
telephone    36.525202
Name: contact, dtype: float64

In [128]:
df.month.value_counts()/df.shape[0]*100

may    33.429640
jul    17.417694
aug    14.999514
jun    12.911528
nov     9.956784
apr     6.390211
oct     1.743226
sep     1.383898
mar     1.325629
dec     0.441876
Name: month, dtype: float64

In [129]:
df.day_of_week.value_counts()/df.shape[0]*100

thu    20.935709
mon    20.671069
wed    19.748470
tue    19.641643
fri    19.003108
Name: day_of_week, dtype: float64

In [130]:
df.groupby(["day_of_week","y"]).education.count()/df.shape[0]*100

day_of_week  y
fri          0    16.949111
             1     2.053996
mon          0    18.614645
             1     2.056424
thu          0    18.398563
             1     2.537147
tue          0    17.327862
             1     2.313781
wed          0    17.444401
             1     2.304069
Name: education, dtype: float64

In [131]:
df.campaign.value_counts()/df.shape[0]*100

1     42.832864
2     25.662814
3     12.967369
4      6.436341
5      3.882199
6      2.376906
7      1.527144
8      0.971157
9      0.687093
10     0.546276
11     0.429737
12     0.303486
13     0.223366
14     0.167525
17     0.140818
15     0.123822
16     0.123822
18     0.080120
20     0.072837
19     0.063125
21     0.058269
22     0.041274
23     0.038846
24     0.036418
27     0.026707
29     0.024279
25     0.019423
26     0.019423
28     0.019423
30     0.016995
31     0.016995
35     0.012139
33     0.009712
32     0.009712
34     0.007284
40     0.004856
42     0.004856
43     0.004856
37     0.002428
39     0.002428
41     0.002428
56     0.002428
Name: campaign, dtype: float64

In [132]:
df.pdays.describe()

count    41188.000000
mean       962.475454
std        186.910907
min          0.000000
25%        999.000000
50%        999.000000
75%        999.000000
max        999.000000
Name: pdays, dtype: float64

In [133]:
df.pdays.value_counts() /df.shape[0]*100

999    96.321744
3       1.065844
6       1.000291
4       0.286491
9       0.155385
2       0.148101
7       0.145673
12      0.140818
10      0.126250
5       0.111683
13      0.087404
11      0.067981
1       0.063125
15      0.058269
14      0.048558
8       0.043702
0       0.036418
16      0.026707
17      0.019423
18      0.016995
19      0.007284
22      0.007284
21      0.004856
26      0.002428
20      0.002428
25      0.002428
27      0.002428
Name: pdays, dtype: float64

In [134]:
df.previous.describe()

count    41188.000000
mean         0.172963
std          0.494901
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          7.000000
Name: previous, dtype: float64

In [135]:
df.poutcome.value_counts()/df.shape[0]*100

nonexistent    86.343110
failure        10.323395
success         3.333495
Name: poutcome, dtype: float64

In [136]:
df["emp.var.rate"].describe()

count    41188.000000
mean         0.081886
std          1.570960
min         -3.400000
25%         -1.800000
50%          1.100000
75%          1.400000
max          1.400000
Name: emp.var.rate, dtype: float64

In [137]:
df["cons.price.idx"].describe()

count    41188.000000
mean        93.575664
std          0.578840
min         92.201000
25%         93.075000
50%         93.749000
75%         93.994000
max         94.767000
Name: cons.price.idx, dtype: float64

In [138]:
df["cons.conf.idx"].describe()

count    41188.000000
mean       -40.502600
std          4.628198
min        -50.800000
25%        -42.700000
50%        -41.800000
75%        -36.400000
max        -26.900000
Name: cons.conf.idx, dtype: float64

In [139]:
df["euribor3m"].describe()

count    41188.000000
mean         3.621291
std          1.734447
min          0.634000
25%          1.344000
50%          4.857000
75%          4.961000
max          5.045000
Name: euribor3m, dtype: float64

In [140]:
df["nr.employed"].describe()

count    41188.000000
mean      5167.035911
std         72.251528
min       4963.600000
25%       5099.100000
50%       5191.000000
75%       5228.100000
max       5228.100000
Name: nr.employed, dtype: float64

Droping duplicates

In [142]:
df = df.drop(df[df.duplicated()].index)

In [143]:
df.dtypes

age                 int64
job                object
marital            object
education          object
housing            object
loan               object
contact            object
month              object
day_of_week        object
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                   int64
dtype: object

Saving clean data

In [144]:
df.to_csv("../data/Clean/bank.csv",sep=';', index=False)

In [145]:
df.head()

,age,job,marital,education,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [146]:
categorical_cols = ["job","marital","education","housing","loan","contact","month","day_of_week","poutcome"]

In [147]:
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [148]:
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [149]:
df.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
0,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,1,0,0,0,1,0,0,0,1,0
1,57,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,1,0,0,0,1,0,0,0,1,0
2,37,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,1,0,0,0,1,0,0,0,1,0
3,40,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,1,0,0,0,1,0,0,0,1,0
4,56,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,...,1,0,0,0,1,0,0,0,1,0


In [150]:
df.to_csv("../data/Clean/bank_encoded.csv", index=False)